In [1]:
import numpy
import timeit
import theano
import theano.tensor as T
import os
import pickle
try:
    import PIL.Image as Image
except ImportError:
    import Image
    
from theano.tensor.shared_randomstreams import RandomStreams

import sys
sys.path.append("../lib")
from load import mnist
from load import faces
from utils import tile_raster_images
from rbm import RBM
from dbn import DBN

Using gpu device 0: GeForce GTX 980


In [2]:
currentDir =  os.getcwd();
parampickle = currentDir + "/parametersDBN.pickle"
logPickle = currentDir + "/errorPickleDBN.pickle"

In [3]:
def test_DBN(finetune_lr=0.01, pretraining_epochs=100,
             pretrain_lr=0.1, k=1, training_epochs=1000,
             dataset='mnist.pkl.gz', batch_size=10):
    """
    Demonstrates how to train and test a Deep Belief Network.

    This is demonstrated on MNIST.

    :type finetune_lr: float
    :param finetune_lr: learning rate used in the finetune stage
    :type pretraining_epochs: int
    :param pretraining_epochs: number of epoch to do pretraining
    :type pretrain_lr: float
    :param pretrain_lr: learning rate to be used during pre-training
    :type k: int
    :param k: number of Gibbs steps in CD/PCD
    :type training_epochs: int
    :param training_epochs: maximal number of iterations ot run the optimizer
    :type dataset: string
    :param dataset: path the the pickled dataset
    :type batch_size: int
    :param batch_size: the size of a minibatch
    """

    #trX, teX, trY, teY  = mnist(onehot=True)
    trX, teX, trY, teY  = faces()
    
    train_set_x = theano.shared(numpy.asarray(trX, dtype=theano.config.floatX),borrow=True)
    test_set_x = theano.shared(numpy.asarray(trX, dtype=theano.config.floatX),borrow=True)
    train_set_y = theano.shared(numpy.asarray(trY, dtype='int32'),borrow=True)
    test_set_y = theano.shared(numpy.asarray(teY, dtype='int32'),borrow=True)
    
    # compute number of minibatches for training, validation and testing
    n_train_batches = train_set_x.get_value(borrow=True).shape[0] / batch_size

    # numpy random generator
    numpy_rng = numpy.random.RandomState(123)
    print '... building the model'
    logline = "... building the model"
    f2 = open(logPickle, 'a+')
    pickle.dump(logline , f2);
    f2.close()
    
    # construct the Deep Belief Network
    dbn = DBN(numpy_rng=numpy_rng, n_ins=48 * 48,
              hidden_layers_sizes=[1000, 1000, 1000],
              n_outs=7)

    # start-snippet-2
    #########################
    # PRETRAINING THE MODEL #
    #########################
    print '... getting the pretraining functions'
    logline = "... getting the pretraining functions"
    f2 = open(logPickle, 'a+')
    pickle.dump(logline , f2);
    f2.close()
    
    pretraining_fns = dbn.pretraining_functions(train_set_x=train_set_x,
                                                batch_size=batch_size,
                                                k=k)

    print '... pre-training the model'
    logline = "... pre-training the model"
    f2 = open(logPickle, 'a+')
    pickle.dump(logline , f2);
    f2.close()
    
    start_time = timeit.default_timer()
    ## Pre-train layer-wise
    for i in xrange(dbn.n_layers):
        # go through pretraining epochs
        for epoch in xrange(pretraining_epochs):
            # go through the training set
            c = []
            for batch_index in xrange(n_train_batches):
                c.append(pretraining_fns[i](index=batch_index,
                                            lr=pretrain_lr))
            print 'Pre-training layer %i, epoch %d, cost ' % (i, epoch),
            print numpy.mean(c)
            
            logline = "Pre-training layer: " + str(i) +"epoch: " + str(epoch) + "mean: " + str(numpy.mean(c))
            f2 = open(logPickle, 'a+')
            pickle.dump(logline , f2);
            f2.close()

    end_time = timeit.default_timer()
    
    ########################
    # FINETUNING THE MODEL #
    ########################

    # get the training, validation and testing function for the model
    print '... getting the finetuning functions'
    logline = "----Getting Finetuning-----"
    f2 = open(logPickle, 'a+')
    pickle.dump(logline , f2);
    f2.close()
    
    train_fn, validate_model, test_model = dbn.build_finetune_functions(
        datasets=datasets,
        batch_size=batch_size,
        learning_rate=finetune_lr
    )

    print '... finetuning the model'
    logline = "------finetuning the model------"
    f2 = open(logPickle, 'a+')
    pickle.dump(logline , f2);
    f2.close()

    # early-stopping parameters
    patience = 4 * n_train_batches  # look as this many examples regardless
    patience_increase = 2.    # wait this much longer when a new best is
                              # found
    improvement_threshold = 0.995  # a relative improvement of this much is
                                   # considered significant
    validation_frequency = min(n_train_batches, patience / 2)
                                  # go through this many
                                  # minibatches before checking the network
                                  # on the validation set; in this case we
                                  # check every epoch

    best_validation_loss = numpy.inf
    test_score = 0.
    start_time = timeit.default_timer()

    done_looping = False
    epoch = 0

    while (epoch < training_epochs) and (not done_looping):
        epoch = epoch + 1
        for minibatch_index in xrange(n_train_batches):

            minibatch_avg_cost = train_fn(minibatch_index)
            iter = (epoch - 1) * n_train_batches + minibatch_index

            if (iter + 1) % validation_frequency == 0:

                validation_losses = validate_model()
                this_validation_loss = numpy.mean(validation_losses)
                print(
                    'epoch %i, minibatch %i/%i, validation error %f %%'
                    % (
                        epoch,
                        minibatch_index + 1,
                        n_train_batches,
                        this_validation_loss * 100.
                    )
                )
                logline = "epoch: " + str(epoch) +"minibatch: " + str(minibatch_index + 1) + "validation error: " + str(this_validation_loss * 100.)
                f2 = open(logPickle, 'a+')
                pickle.dump(logline , f2);
                f2.close()

                # if we got the best validation score until now
                if this_validation_loss < best_validation_loss:

                    #improve patience if loss improvement is good enough
                    if (
                        this_validation_loss < best_validation_loss *
                        improvement_threshold
                    ):
                        patience = max(patience, iter * patience_increase)

                    # save best validation score and iteration number
                    best_validation_loss = this_validation_loss
                    best_iter = iter

                    # test it on the test set
                    test_losses = test_model()
                    test_score = numpy.mean(test_losses)
                    print(('     epoch %i, minibatch %i/%i, test error of '
                           'best model %f %%') %
                          (epoch, minibatch_index + 1, n_train_batches,
                           test_score * 100.))
                    logline = "epoch: " + str(epoch) +"minibatch: " + str(minibatch_index + 1) + "Test error: " + str(test_score * 100.)
                    f2 = open(logPickle, 'a+')
                    pickle.dump(logline , f2);
                    f2.close()


            if patience <= iter:
                done_looping = True
                break

    end_time = timeit.default_timer()
    
    print(
        (
            'Optimization complete with best validation score of %f %%, '
            'obtained at iteration %i, '
            'with test performance %f %%'
        ) % (best_validation_loss * 100., best_iter + 1, test_score * 100.))
    
    logline = "Optimization complete with best validation score of: " + str(best_validation_loss * 100.) +"obtained at iteration: " + str(i) + "Test error: " + str(test_score * 100.)
    f2 = open(logPickle, 'a+')
    pickle.dump(logline , f2);
    f2.close()
    
    dbn.save_DBN_state(parampickle)

  

In [ ]:
test_DBN()